## Loading the MEE data and the associated metadata

In [1]:
import pandas as pd
import numpy as np
from mab2rec.utils import print_interaction_stats
import matplotlib.pyplot as plt
import seaborn as sns
import json

sns.set_style("whitegrid")

In [ ]:
dir_data = "/home/tlefort/Documents/phd/mee_paper/created_data/"
with open(dir_data+"species_to_families_genus.json", "r") as f:
    species_to_upper = json.load(f)
with open(dir_data+"answers.json", "r") as f:
    answers = json.load(f)
with open(dir_data+"metadata.json", "r") as f:
    metadata = json.load(f)